In [9]:
import optuna
from pysr import PySRRegressor
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
# 1. Prepare toy data
X = np.random.randn(200, 5)
y = 2.5 * np.cos(X[:, 2]) + X[:, 0]**2 - 0.3 * X[:, 1]

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 2. Define Optuna objective
def objective(trial: optuna.Trial) -> float:
    # Suggest PySR hyperparameters
    pop_size = trial.suggest_int("population_size", 50, 500)
    n_iters = trial.suggest_int("niterations", 10, 100)
    parsimony = trial.suggest_float("parsimony", 0.001, 0.1, log=True)
    optimizer_restarts = trial.suggest_int("optimizer_nrestarts", 1, 10)

    model = PySRRegressor(
        population_size=pop_size,
        niterations=n_iters,
        parsimony=parsimony,
        optimizer_nrestarts=optimizer_restarts,
        # you can add more PySR settings here...
    )

    # Fit and evaluate
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)
    return mean_squared_error(y_valid, y_pred)

# 3. Create and run the study
sampler = optuna.samplers.TPESampler()  # default TPE sampler :contentReference[oaicite:2]{index=2}
pruner = optuna.pruners.MedianPruner()  # stop unpromising trials early :contentReference[oaicite:3]{index=3}

study = optuna.create_study(
    storage = "sqlite:///db.sqlite3",
    study_name = "pysr-test_study",
    direction="minimize",
    sampler=sampler,
    pruner=pruner
)
study.optimize(objective, n_trials=50, timeout=600)

print("Best MSE:", study.best_value)
print("Best params:", study.best_params)

In [3]:
study.best_params

{'population_size': 407,
 'niterations': 77,
 'parsimony': 0.08919810761494844,
 'optimizer_nrestarts': 6}

# Multiple Optimization - Pareto Front

In [10]:
import numpy as np

# Set a random seed for reproducibility
np.random.seed(42)

# Create 5 datasets
Xs = []
Ys = []

n_samples = 100  # Number of points per dataset

# Dataset 1: Linear
X1 = np.random.uniform(-5, 5, size=(n_samples, 1))
y1 = 2 * X1[:, 0] + 1 + np.random.normal(0, 0.5, size=n_samples)

# Dataset 2: Quadratic
X2 = np.random.uniform(-3, 3, size=(n_samples, 1))
y2 = 0.5 * X2[:, 0]**2 - 3 * X2[:, 0] + 2 + np.random.normal(0, 0.5, size=n_samples)

# Dataset 3: Sinusoidal
X3 = np.random.uniform(0, 2*np.pi, size=(n_samples, 1))
y3 = np.sin(X3[:, 0]) + 0.1 * np.random.normal(0, 1, size=n_samples)

# Dataset 4: Exponential decay
X4 = np.random.uniform(0, 5, size=(n_samples, 1))
y4 = np.exp(-X4[:, 0]) + np.random.normal(0, 0.05, size=n_samples)

# Dataset 5: Cubic
X5 = np.random.uniform(-2, 2, size=(n_samples, 1))
y5 = X5[:, 0]**3 - X5[:, 0]**2 + X5[:, 0] + np.random.normal(0, 0.2, size=n_samples)

# Store them in lists
Xs = [X1, X2, X3, X4, X5]
Ys = [y1, y2, y3, y4, y5]


In [11]:
import numpy as np
from sklearn.metrics import mean_squared_error
import optuna
from optuna.samplers import NSGAIIISampler
from pysr import PySRRegressor

# Assume you have 5 datasets in lists Xs and Ys
Xs = [X1, X2, X3, X4, X5]
Ys = [y1, y2, y3, y4, y5]

def objective(trial):
    # Suggest shared hyperparameters
    pop_size = trial.suggest_int("population_size", 50, 500)
    n_iters = trial.suggest_int("niterations", 10, 100)
    parsimony = trial.suggest_float("parsimony", 1e-4, 1e-1, log=True)

    # Instantiate PySR with these hyperparameters
    model = PySRRegressor(
        population_size=pop_size,
        niterations=n_iters,
        parsimony=parsimony
    )

    # Compute MSE on each dataset
    losses = []
    for X, y in zip(Xs, Ys):
        model.fit(X, y)
        losses.append(mean_squared_error(y, model.predict(X)))

    # Return a tuple of five objectives (one per dataset)
    return tuple(losses)

# Create a multi-objective study for 5 minimization tasks
study = optuna.create_study(
    storage = "sqlite:///db.sqlite3",
    study_name = "pysr_Pareto_III-test_study",
    directions=["minimize"] * 5,
    sampler=NSGAIIISampler()
)

# Run 100 trials of the genetic algorithm
study.optimize(objective, n_trials=100, timeout=600)

# Retrieve the Pareto front trials
pareto_trials = study.best_trials
print("Number of Pareto-optimal trials:", len(pareto_trials))


C:\Users\guilherme\AppData\Local\Temp\ipykernel_8344\915575966.py:38: ExperimentalWarning: NSGAIIISampler is experimental (supported from v3.2.0). The interface can change in the future.
  sampler=NSGAIIISampler()
[I 2025-04-27 16:45:14,314] A new study created in RDB with name: pysr_Pareto_III-test_study
e:\Documents\Coding\Repositorios\MyRepositories\NewMortalityCurves\.venv\Lib\site-packages\pysr\sr.py:2774: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
Compiling Julia backend...
[ Info: Started!



Expressions evaluated per second: 1.430e+05
Progress: 86 / 1674 total iterations (5.137%)
════════════════════════════════════════════════════════════════════════════════════════════════════
───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           9.057e+00  1.594e+01  y = x₀
3           1.177e+00  1.020e+00  y = x₀ * 1.9436
5           2.063e-01  8.708e-01  y = x₀ + (x₀ - -0.99946)
7           2.016e-01  1.135e-02  y = ((x₀ + x₀) - -1.0042) * 0.9885
9           2.013e-01  9.065e-04  y = x₀ + ((-51.141 / (-51.495 - x₀)) + x₀)
13          1.978e-01  4.420e-03  y = x₀ + (x₀ + (x₀ / (x₀ + ((0.0056928 / x₀) - -0.031135))...
                                      ))
15          1.974e-01  1.003e-03  y = (x₀ + ((x₀ / ((x₀ + (0.004895 / x₀)) - -0.030506)) + x...
                                      ₀)) + 0.020047
17          1.943e-01  7.809e-03  y = -1.6208 - (x₀ + (-0.41887 - ((x₀ + 2.0926) - 

[ Info: Final population:
[ Info: Results saved to:
e:\Documents\Coding\Repositorios\MyRepositories\NewMortalityCurves\.venv\Lib\site-packages\pysr\sr.py:2191: UserWarning: The discovered expressions are being reset. Please set `warm_start=True` if you wish to continue to start a search where you left off.
  warnings.warn(
e:\Documents\Coding\Repositorios\MyRepositories\NewMortalityCurves\.venv\Lib\site-packages\pysr\sr.py:2774: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
[ Info: Started!



Expressions evaluated per second: 1.660e+05
Progress: 97 / 1674 total iterations (5.795%)
════════════════════════════════════════════════════════════════════════════════════════════════════
───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           2.775e+01  1.594e+01  y = 2.9861
3           1.331e+01  3.675e-01  y = 3.1561 - x₀
5           2.106e+00  9.217e-01  y = 3.4867 - (x₀ * 2.9487)
7           2.043e+00  1.531e-02  y = (x₀ * 0.8607) * (x₀ + -3.4205)
9           1.987e-01  1.165e+00  y = (((x₀ * 0.50077) + -2.9769) * x₀) - -1.9998
17          1.930e-01  3.644e-03  y = (x₀ * (((2.3207 - (x₀ * 0.085695)) * ((x₀ / -1.2351) *...
                                       -0.26728)) - 2.874)) + 1.9904
19          1.921e-01  2.419e-03  y = (x₀ * ((1.0316 - ((x₀ * x₀) * -0.0069491)) * (((x₀ / -...
                                      3.3927) * -1.5712) - 2.7826))) + 2.0107
23          1.907e-

[ Info: Final population:
[ Info: Results saved to:


...

───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           4.147e-02  1.594e+01  y = 0.17636
3           1.003e-02  7.097e-01  y = 0.23144 / x₀
5           5.667e-03  2.855e-01  y = 0.35439 / (x₀ + 0.18436)
7           2.900e-03  3.349e-01  y = 0.5848 / ((x₀ * x₀) + 0.70394)
9           2.869e-03  5.396e-03  y = 0.66899 / ((x₀ * (x₀ + 0.22322)) + 0.74355)
11          2.780e-03  1.570e-02  y = 2.1209 / ((x₀ * ((x₀ * x₀) - -3.1024)) + 1.9365)
13          2.779e-03  1.840e-04  y = (1.9654 / ((((x₀ * x₀) - -2.8044) * x₀) + 1.8264)) + 0...
                                      .0027644
15          2.690e-03  1.633e-02  y = 0.18883 / (((x₀ * x₀) / ((0.049011 / (x₀ + -0.98684)) ...
                                      + 3.0688)) + 0.22374)
17          2.681e-03  1.574e-03  y = (0.19651 / ((x₀ * (x₀ / ((0.051073 / (x₀ + -0.98513)) ...
                                      + 3.0665))) + 0.23291)) + 

e:\Documents\Coding\Repositorios\MyRepositories\NewMortalityCurves\.venv\Lib\site-packages\pysr\sr.py:2191: UserWarning: The discovered expressions are being reset. Please set `warm_start=True` if you wish to continue to start a search where you left off.
  warnings.warn(
e:\Documents\Coding\Repositorios\MyRepositories\NewMortalityCurves\.venv\Lib\site-packages\pysr\sr.py:2774: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
[ Info: Started!



Expressions evaluated per second: 1.100e+05
Progress: 80 / 1767 total iterations (4.527%)
════════════════════════════════════════════════════════════════════════════════════════════════════
───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           1.248e+01  1.594e+01  y = x₀
3           4.618e+00  4.972e-01  y = x₀ / 0.29172
5           2.741e+00  2.608e-01  y = (x₀ / 0.29332) + -1.3701
7           1.357e+00  3.515e-01  y = x₀ * ((x₀ + -1.0152) * x₀)
9           3.432e-02  1.839e+00  y = ((x₀ * x₀) * (x₀ + -0.9757)) + x₀
11          3.408e-02  3.483e-03  y = (x₀ * (((x₀ + -0.98572) * x₀) + 0.99697)) + 0.023168
13          3.408e-02  2.086e-07  y = ((((x₀ + -0.36157) * x₀) + 0.77129) * (x₀ + -0.62416))...
                                       + 0.50457
17          3.399e-02  6.496e-04  y = (((x₀ + -0.94675) * ((x₀ * x₀) + ((x₀ + 0.55652) * -0....
                                      046

[ Info: Final population:
[ Info: Results saved to:
[I 2025-04-27 16:59:34,195] Trial 1 finished with values: [0.20164614100529488, 0.198699930159693, 0.010874073796885278, 0.0029000965712195443, 0.03430928231042018] and parameters: {'population_size': 227, 'niterations': 57, 'parsimony': 0.000520657623421051}.


Number of Pareto-optimal trials: 2
  - outputs\20250427_165803_ZVbc7X\hall_of_fame.csv


In [5]:
study.best_trials

[FrozenTrial(number=0, state=1, values=[0.20164614099179637, 0.19869994477082342, 0.009669461172165314, 0.002900097516160072, 0.034309282455505505], datetime_start=datetime.datetime(2025, 4, 27, 12, 54, 2, 630152), datetime_complete=datetime.datetime(2025, 4, 27, 12, 56, 29, 504019), params={'population_size': 201, 'niterations': 29, 'parsimony': 0.0951050183328403}, user_attrs={}, system_attrs={'NSGAIISampler:generation': 0}, intermediate_values={}, distributions={'population_size': IntDistribution(high=500, log=False, low=50, step=1), 'niterations': IntDistribution(high=100, log=False, low=10, step=1), 'parsimony': FloatDistribution(high=0.1, log=True, low=0.0001, step=None)}, trial_id=8, value=None),
 FrozenTrial(number=1, state=1, values=[0.20164614112240606, 0.19869993015703405, 0.009837724863500975, 0.0029000971647151538, 0.034309282175158134], datetime_start=datetime.datetime(2025, 4, 27, 12, 56, 29, 964146), datetime_complete=datetime.datetime(2025, 4, 27, 13, 2, 6, 841446), pa

In [13]:
import pickle

with open("test_pareto_study_III.pkl", "rb") as file:
    # pickle.dump(study, file)
    study = pickle.load(file)
    

In [14]:
study.best_trials

[FrozenTrial(number=0, state=1, values=[0.20164614180977355, 0.198699930160252, 0.010874067896239903, 0.002900096577527691, 0.034309282287740606], datetime_start=datetime.datetime(2025, 4, 27, 16, 45, 14, 325475), datetime_complete=datetime.datetime(2025, 4, 27, 16, 52, 28, 830727), params={'population_size': 314, 'niterations': 54, 'parsimony': 0.002595596720701991}, user_attrs={}, system_attrs={'nsga3:generation': 0}, intermediate_values={}, distributions={'population_size': IntDistribution(high=500, log=False, low=50, step=1), 'niterations': IntDistribution(high=100, log=False, low=10, step=1), 'parsimony': FloatDistribution(high=0.1, log=True, low=0.0001, step=None)}, trial_id=11, value=None),
 FrozenTrial(number=1, state=1, values=[0.20164614100529488, 0.198699930159693, 0.010874073796885278, 0.0029000965712195443, 0.03430928231042018], datetime_start=datetime.datetime(2025, 4, 27, 16, 52, 29, 285464), datetime_complete=datetime.datetime(2025, 4, 27, 16, 59, 33, 774147), params={'

In [15]:
optuna.visualization.plot_pareto_front(study, 
    targets=lambda t: (t.values[0], t.values[1]),
    target_names=["Objective 0", "Objective 1"]
)

In [ ]:
import numpy as np
from pymoo.indicators.hv import Hypervolume
from kneed import KneeLocator

# Suppose `pareto_vals` is your (n_points × 5) NumPy array of objective values
pareto_vals = np.array([t.values for t in study.best_trials])  # shape (n_points, 5)

# 1. Normalize objectives to [0,1]
ideal = pareto_vals.min(axis=0)
nadir = pareto_vals.max(axis=0)
norm = (pareto_vals - ideal) / (nadir - ideal + 1e-12)

# 2. Hypervolume contribution
ref_point = np.ones(5)  # reference outside normalized front
hv_func = Hypervolume(ref_point=ref_point)
hv_all = hv_func(norm)
# compute contribution for each point
contr = np.zeros(norm.shape[0])
for i in range(norm.shape[0]):
    mask = np.arange(norm.shape[0]) != i
    contr[i] = hv_all - hv_func(norm[mask])
best_hv_idx = np.argmax(contr)

# 3. Knee detection on sorted Manhattan distances
manh = np.sum(norm, axis=1)
sorted_idx = np.argsort(manh)
sorted_manh = manh[sorted_idx]
kl = KneeLocator(
    x=list(range(len(sorted_manh)))
    y=sorted_manh.tolist(),
    curve="concave",
    direction="increasing"
)
knee_rank = kl.knee  # index in sorted list
best_knee_idx = sorted_idx[knee_rank]


# 5. Aggregate or compare
print("Best by hypervolume contribution:", best_hv_idx)
print("Best by knee detection:", best_knee_idx)


Best by hypervolume contribution: 2
Best by knee detection: [[2 1 0]]


In [8]:
pareto_vals[2]

array([0.20164644, 0.19869993, 0.00941338, 0.0029001 , 0.03430928])